# Import

In [1]:
import numpy as np

# 1. UPGMA :

# Variables

In [2]:
                            #A,B,C,D,E,F  
distance_matrix = np.array([[0,6,4,6,8,4], #A
                            [6,0,6,4,8,6], #B
                            [4,6,0,6,8,2], #C
                            [6,4,6,0,8,6], #D
                            [8,8,8,8,0,8], #E
                            [4,6,2,6,8,0]])#F

In [3]:
class Tree:
    def __init__(self):
        self.left = None
        self.right = None
        self.species1 = ""
        self.species2 = ""
        self.distance = 0
    def add_nodes(self, min_v, v1, v2):
        self.left = Tree()
        self.right = Tree()
        self.name = v1+v2
        self.species1 = v1
        self.species2 = v2
        self.distance = min_v
    def display_tree(self, node):
        if node is not None:
            print("   __")
            print("  |  |")
            print(node.distance, "|  |", node.distance)
            print("  |  |")
            print(" ", node.species1, "", node.species2)
            if node.left is not None:
                self.display_tree(self.left)
            if node.right is not None:
                self.display_tree(self.right)

In [4]:
def upgma_loop(distance_matrix, dic):
    cpt = 1
    while distance_matrix.shape[0] > 3:
        print("======================== Nombre d'itération :", cpt,"========================")
        display_distance_matrix(distance_matrix, dic)
        print("\nDictionnaire :\n", dic)
        distance_matrix, dic = upgma(distance_matrix, dic)
        cpt += 1
    return distance_matrix, dic

def upgma(distance_matrix, dic):
    tree = Tree()
    min_v, v1, v2 = get_min_v(distance_matrix, dic)
    print('v1',v1)
    print("v2",v2)
    tree.add_nodes(min_v, v1, v2)
    distance_matrix_u = update_distance_matrix(distance_matrix, v1, v2, dic)
    distance_matrix_u = distance_matrix_u.astype(int)
    dic_u = update_dic(dic, v1, v2)
    return distance_matrix_u, dic_u

def get_min_v(distance_matrix, dic):
    n,m = distance_matrix.shape
    min_v = np.inf
    for i in range(n):
        for j in range(m):
            if i != j: # Ignore diagonal
                if min_v > distance_matrix[i][j]:
                    min_v = distance_matrix[i][j]
                    index_v1 = i
                    index_v2 = j
    return min_v, dic[index_v1], dic[index_v2]

def update_dic(dic, v1, v2):
    new_val = v1+v2
    dic_u = {}
    n = len(dic) - 1
    for i in range(n):
        if chr(i+65) != v1 and chr(i+65) != v2:
            keys = [chr(i+65) for i in range(n)]
            dic_u[i] = keys[i]
    dic_u[n] = new_val
    return dic_u

def update_distance_matrix(distance_matrix, v1, v2, dic):
    k1 = [k for k, v in dic.items() if v == v1][0]
    distance_matrix_u = np.delete(distance_matrix, k1, 0)   # Delete ligne à l'indice k1 correspondant à v1
    distance_matrix_u = np.delete(distance_matrix_u, k1, 1) # Delete colonne à l'indice k1 correspondant à v1
    new_distances = compute_new_distances(distance_matrix, v1, v2, dic)
    distance_matrix_u[-1] = new_distances                   # Ajout des valeurs calculées sur la derniere ligne
    distance_matrix_u[:,-1] = new_distances.T               # et la derniere colonne correspondant à {v1,v2}
    return distance_matrix_u

def compute_new_distances(distance_matrix, v1, v2, dic):
    n = distance_matrix.shape[0]
    new_distances = np.zeros((n-1))
    k1 = [k for k, v in dic.items() if v == v1][0]
    k2 = [k for k, v in dic.items() if v == v2][0]
    decal = 0
    for i in range(n-1):
        if i == k1 or i == k2: # Si on est sur une ligne de v1 on doit decaler
            decal += 1
            decal = True
            new_distances[i] = (distance_matrix[i+decal][k1] + distance_matrix[i+decal][k2]) / 2
        elif decal == True:    # Si on est pas sur une ligne mais qu'on a fait un decalager on doit decaler
            new_distances[i] = (distance_matrix[i+decal][k1] + distance_matrix[i+decal][k2]) / 2
        else:                  # Autrement on fait la distance normale
            new_distances[i] = (distance_matrix[i][k1] + distance_matrix[i][k2]) / 2
    new_distances[-1] = 0      # Remise à 0 de la diagonal
    return new_distances

def display_distance_matrix(distance_matrix, dic):
    print("Distance matrix :\n")
    n,m = distance_matrix.shape
    for val in dic.values():
        print("",val, end="")
    print("")
    for i,val in enumerate(dic.values()):
        for j in range(m):
            print("", distance_matrix[i][j], end="")
        print("", val)

def get_dict(distance_matrix):
    dic = {}
    n = distance_matrix.shape[0]
    keys = [chr(i+65) for i in range(n)]
    values = [i for i in range(n)]
    for i in range(n):
        dic[values[i]] = keys[i]
    return dic

In [5]:
dic = get_dict(distance_matrix)

# distance_matrix_u, dic_u = upgma(distance_matrix, dic)
# display_distance_matrix(distance_matrix_u, dic_u)

distance_matrix_final, dic_final = upgma_loop(distance_matrix, dic)

======================== Nombre d'itération : 1 ========================
Distance matrix :

 A B C D E F
 0 6 4 6 8 4 A
 6 0 6 4 8 6 B
 4 6 0 6 8 2 C
 6 4 6 0 8 6 D
 8 8 8 8 0 8 E
 4 6 2 6 8 0 F

Dictionnaire :
 {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F'}
v1 C
v2 F
======================== Nombre d'itération : 2 ========================
Distance matrix :

 A B D E CF
 0 6 6 8 4 A
 6 0 4 8 6 B
 6 4 0 8 6 D
 8 8 8 0 8 E
 4 6 6 8 0 CF

Dictionnaire :
 {0: 'A', 1: 'B', 3: 'D', 4: 'E', 5: 'CF'}
v1 A
v2 E
======================== Nombre d'itération : 3 ========================
Distance matrix :

 B C D AE
 0 4 8 6 B
 4 0 8 6 C
 8 8 0 8 D
 6 6 8 0 AE

Dictionnaire :
 {1: 'B', 2: 'C', 3: 'D', 4: 'AE'}


KeyError: 0

# 2. Neighbor Joining :